In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs

In [ ]:
import matplotlib.patches as mpatches

In [ ]:
tempMax = xr.open_dataset("/home/mlopez/EXEC/Données test/tasmax_day_BNU-ESM_rcp45_r1i1p1_na10kgrid_qm-moving-50bins-detrend_1956.nc")

In [ ]:
tempMax

In [ ]:
#Not useful, tasmax is all NAN and data is not well distributed. Process is more complex. 
#Make same as R, first, transform into vector and matrix, then DF
df = tempMax.to_dataframe().dropna()
df.head

# Plotting Timeseries

In [ ]:
AllcoordTempMean = df.groupby("time").mean()
AllcoordTempMean

In [ ]:
TempMeanC = AllcoordTempMean
TempMeanC["tasmaxC"] = TempMeanC["tasmax"] -273.15
TempMeanC.head()


In [ ]:
# plot time series
TempMeanC['tasmaxC'].plot.line('o-',color='blue',figsize=(15,10))

# add colour reference
blue_patch = mpatches.Patch(color='blue', label='Temperature Maximale')
plt.legend(handles=[blue_patch,])

# add titles
plt.xlabel('Date')
plt.ylabel('Temperature (C)')
plt.title('Temperature Maximale')


In [ ]:
AlltimeTempMean = df.groupby(["lat","lon"]).mean()
AlltimeTempMean

In [ ]:
AlltimeTempMean["tasmaxC"] = GeoTempMeanC["tasmax"] -273.15

In [ ]:
AlltimeTempMean

In [ ]:
GeoTempMeanC = AlltimeTempMean.reset_index()

In [ ]:
GeoTempMeanC["tasmaxC"]

In [ ]:
GeoTempMeanC["lat"]

In [ ]:
plt.pcolor(GeoTempMeanC)
plt.yticks(np.arange(0.5, len(GeoTempMeanC.lon), 1), GeoTempMeanC.lon)
plt.xticks(np.arange(0.5, len(GeoTempMeanC.lat), 1), GeoTempMeanC.lat)
plt.show()

In [ ]:
GeoTempMeanC.plot(kind="scatter", x="lon", y="lat",
    c="tasmaxC", cmap=plt.get_cmap("jet"),
    colorbar=True, alpha=0.4, figsize=(10,7),
)
plt.legend()
plt.show()

In [ ]:
lon = tempMax.variables['lon'][:]
lon

In [ ]:
lat = tempMax.variables['lat'][:]
lat

In [ ]:
tasmax = tempMax.variables['tasmax'][:]
time = tempMax.variables['time']
time


In [ ]:
tasmax

In [ ]:
tempMax.variables

In [ ]:
tempMax.values 

In [ ]:
tempMax.dims

In [ ]:
tempMax.coords

In [ ]:
tempMax.attrs

In [ ]:
# Extract Temperature 
# Select nearest
# Drop NaN, convert to Celcius
da = tempMax.tasmax.sel(method='nearest').dropna(dim='time') - 273.15  # or .ffill(dim='time')

In [ ]:
da

In [ ]:
da_numpy = tempMax.values
da_numpy